<a href="https://colab.research.google.com/github/tr33h/main_notebook/blob/main/main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create config file

In [31]:
from configparser import ConfigParser

#Get the configparser object
config = ConfigParser()

#Assume we need 2 sections in the config file, let's call them USERINFO and SERVERCONFIG
config["TWTOKEN"] = {
    "consumer_key": "",
    "consumer_secret_key": "",
    "access_token": "",
    "access_token_secret": ""
}

#Write the above sections to config.ini file
with open('config.ini', 'w') as conf:
    config.write(conf)

# Initialize and authenticate

In [32]:
# Import dependencies
import openpyxl
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime
import os
from configparser import ConfigParser

In [34]:
# Read config.ini file
config = ConfigParser()
config.read('config.ini')

consumer_key = config.get('TWTOKEN', 'consumer_key')
consumer_secret_key = config.get('TWTOKEN', 'consumer_secret_key')
access_token = config.get('TWTOKEN', 'access_token')
access_token_secret = config.get('TWTOKEN', 'access_token_secret')

In [36]:
# Sign in API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [41]:
try:
  auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to authenticate.')

Error! Failed to authenticate.


In [5]:
# Create datastructure
df = pd.DataFrame(columns = ['Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date'])

# Define function
def search(data):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
    #  if (not tweet.retweeted) and ('RT @' not in tweet.text):                  # removes retweets
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc [i, 'tweet_date'] = tweet.created_at
        df.to_csv('Log_rawdata.csv', index = False)
        i+=1
        if i == 1000:
            break
        else:
            pass

# Search function

In [ ]:
# Search function
search(data = ['#medtech'or '#lifescience'or '#healthcare'or '#pharma'or '#ai'or '#Artificialintelligence'or'#machinelearning'or'#ML'or'#pharmaceuticals','#medicaldevices'or'#diagnostics'or'#nlp'or'#reinforcementlearning'or'#computervision'or'#robotics' ])


# Clean search result
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())
df['Tweets'] = df['Tweets'].apply(lambda x: clean_tweet(x))

print(str(len(df.index))+" Tweets found!")

# Output

In [ ]:
# Show data
df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# save csv file
if not os.path.isfile('Hashtag Docu_'+datetime.now().strftime("%Y_%m_%d")+'.csv'):
   df.to_csv('Hashtag Docu_'+datetime.now().strftime("%Y_%m_%d")+'.csv', header='column_names')
else: # else it exists so append without writing the header
   df.to_csv('Hashtag Docu_'+datetime.now().strftime("%Y_%m_%d")+'.csv', mode='a', header=False)

In [ ]:
# delete csv file
try:
  os.remove('Hashtag Docu_'+datetime.now().strftime("%Y_%m_%d")+'.csv')
  print("The file: {} is deleted!".format(my_file))
except OSError as e:
    print("Error: {} - {}!".format(e.filename, e.strerror))

In [ ]:
# download csv file
try:
  from google.colab import files
  files.download('Hashtag Docu_'+datetime.now().strftime("%Y_%m_%d")+'.csv')
except:
  print("Error: No such file or directory!")